<a href="https://colab.research.google.com/github/cindyy-chen/iss_movement/blob/main/manipulating_ISS_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of the ISS Movement

## Required packages
- Matplotlib
- NumPy
- Pandas
- Cartopy (might not be needed at the end)
- Shapely
- GeoPandas
- MovingPandas
- HoloViews
- hvplot

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
from pathlib import Path
import datetime as dt

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [ ]:
import cartopy
import cartopy.crs as ccrs

In [ ]:
from shapely import geometry as shpgeom
from shapely import wkt as shpwkt

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd

In [ ]:
import holoviews as hv
import hvplot.pandas

In [ ]:
plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
hv.opts.defaults(hv.opts.Overlay(active_tools=['wheel_zoom'],
                                 frame_width=500, frame_height=400))
hvplot_defaults = {'tiles':None, 'cmap':'Viridis', 'colorbar':True}

In [ ]:
mpd.show_versions()

## Step 1: Read the CSV File

In [ ]:
file_name = "iss_timeseries_trajectories_20240211_165733.csv"

In [ ]:
df = pd.read_csv(file_name)

In [ ]:
df

In [ ]:
df.info()

Note that the column `t` has string as data type. We want it to be a datetime object.

## Step 2: Perform Conversions

#### Convert the `t` column into a datetime object

In [ ]:
df['t'] = pd.to_datetime(df['t'], format='%Y-%m-%d %H:%M:%S')

Make the `t` column as index

In [ ]:
df.set_index('t', inplace=True)

In [ ]:
df.head(4)

#### Count the number of locations that are on land:
- `land_flag` is `True` for land and `False` otherwise (ocean).

In [ ]:
df.land_flag.sum()

#### Convert the positions (latitude and longitude) into `Shapely` POINT objects:
- Add a `geometry` column

In [ ]:
df['geometry'] = [shpgeom.Point(xy) for xy in zip(df['longitude'], df['latitude'])]
df

In [ ]:
df.info()

In [ ]:
df.head(4)

## Step 3: Manipulation with GeoPandas

#### Create a GeoDataFrame

In [ ]:
iss_gdf = gpd.GeoDataFrame(df, geometry="geometry")
iss_gdf

#### Quick plot

In [ ]:
iss_gdf.plot()

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': ccrs.PlateCarree()})
fig.set_size_inches(10, 10)
iss_gdf.plot(
    ax=ax
)

ax.coastlines()
plt.title('Movement of ISS', fontsize = 14);

#### Plot by country crossed using Cartopy

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': ccrs.PlateCarree()})
fig.set_size_inches(10, 10)
iss_gdf.plot("country", ax=ax)

ax.coastlines()
ax.set_global()
plt.title('Movement of ISS by Country', fontsize = 14);

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': ccrs.PlateCarree()})
fig.set_size_inches(10, 10)
iss_gdf[iss_gdf.land_flag==True].plot("country", ax=ax)

ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
ax.coastlines()
ax.set_global()
plt.title('Movement of ISS by Country', fontsize = 14);

#### Plot by country crossed using the Natural Earth Database background image

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10))
base = world.plot(ax=ax, color='white', edgecolor='black',)

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10))
base = world.plot(ax=ax, color='white', edgecolor='black',)
iss_gdf.plot(ax=base, marker="*", markersize=50, cmap = 'jet');

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10))
base = world.plot(ax=ax, color='white', edgecolor='black',)
iss_gdf[iss_gdf.land_flag==True].plot("country", ax=base);

# Step 4: Manipulation with MovingPandas

#### Create the MovingPandas trajectories

Create a GeoDataFram by specifying the Coordinate Reference System (CRS)

In [ ]:
gdf_path = gpd.GeoDataFrame(df, crs={'init': 'epsg:4326'})

Create the MovingPandas trajectories

In [ ]:
iss_trajc = mpd.TrajectoryCollection(gdf_path,
                                 x = "longitude", y="latitude",
                                 traj_id_col="traj_id", t="t")

#### Quick plots

Simple plot with all the trajectories

In [ ]:
iss_trajc.plot(column='traj_id', figsize=(10,20));

Determine the number of trajectories

In [ ]:
num_trajs = len(iss_trajc.trajectories)
print(f"Number of trajectories: {num_trajs}")

Plot the individual trajectories

In [ ]:
colors = ["green", "red", "blue", "purple"]
fig, ax = plt.subplots(num_trajs, 1)
for i in range(num_trajs):
    iss_trajc.trajectories[i].plot(ax=ax[i], color=colors[i])

#### Include Weather Data

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10))
base = world.plot(ax=ax, color='white', edgecolor='black',)
iss_trajc.trajectories[0].plot(ax=base, legend="true", column="temperature", cmap='jet')

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10))
base = world.plot(ax=ax, color='white', edgecolor='black',)
iss_trajc.trajectories[1].plot(ax=base, legend="true", column="temperature", cmap='jet')

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10))
base = world.plot(ax=ax, color='white', edgecolor='black',)
iss_trajc.trajectories[2].plot(ax=base, legend="true", column="temperature", cmap='jet')

#### Create interactive plots with weather data

In [ ]:
iss_trajc.trajectories[1].hvplot(c="windspeed", tiles="OSM")

In [ ]:
gdf_path2 = gpd.GeoDataFrame(df[df.land_flag==True], crs={'init': 'epsg:4326'})
iss_trajc2 = mpd.TrajectoryCollection(gdf_path2,
                                 x = "longitude", y="latitude",
                                 traj_id_col="traj_id", t="t")

In [ ]:
iss_trajc2.plot(column='traj_id', figsize=(10,20));